In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [16]:
!pip install split-folders

In [17]:
import splitfolders

input_folder = "/kaggle/input/plantdisease/PlantVillage"
output_folder = "/kaggle/working/PlantVillage_Split"

splitfolders.ratio(
    input_folder,
    output=output_folder,
    seed=42,
    ratio=(.8, .1, .1)   # train, val, test
)

Copying files: 20639 files [03:25, 100.30 files/s]


In [43]:
import glob
import os

folders = glob.glob("/kaggle/input/plantdisease/PlantVillage/*")
folder_names = sorted(os.path.basename(f) for f in folders)

folder_names



['Pepper__bell___Bacterial_spot',
 'Pepper__bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [20]:
img=256
b_s=32
train_datagen= ImageDataGenerator(
    rescale=1/255.,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

In [21]:
valid_test_datagen=ImageDataGenerator(rescale=1/255.)

In [18]:
train_dir="/kaggle/working/PlantVillage_Split/train"
valid_dir="/kaggle/working/PlantVillage_Split/val"
test_dir="/kaggle/working/PlantVillage_Split/test"

In [55]:
train=train_datagen.flow_from_directory(
    train_dir,
    target_size=(img,img),
    batch_size=b_s,
    class_mode="categorical",
    shuffle=True,
    seed=123
)

Found 16504 images belonging to 15 classes.


In [56]:
train[0]

(array([[[[0.7348758 , 0.73095423, 0.7858562 ],
          [0.7457799 , 0.7418583 , 0.79676026],
          [0.75906706, 0.7551455 , 0.81004745],
          ...,
          [0.6627451 , 0.654902  , 0.7058824 ],
          [0.67939025, 0.6715471 , 0.7225275 ],
          [0.6736791 , 0.665836  , 0.71681637]],
 
         [[0.7499366 , 0.746015  , 0.80091697],
          [0.75082725, 0.7469057 , 0.80180764],
          [0.7501231 , 0.7462015 , 0.8011035 ],
          ...,
          [0.6627451 , 0.654902  , 0.7058824 ],
          [0.68060756, 0.6727644 , 0.7237448 ],
          [0.67266464, 0.6648215 , 0.71580195]],
 
         [[0.75982946, 0.7559079 , 0.81080985],
          [0.75421417, 0.7502926 , 0.80519456],
          [0.74605274, 0.7421312 , 0.79703313],
          ...,
          [0.6627451 , 0.654902  , 0.7058824 ],
          [0.68182486, 0.6739817 , 0.7249621 ],
          [0.6716503 , 0.66380715, 0.71478754]],
 
         ...,
 
         [[0.57853955, 0.56819266, 0.6204249 ],
          [0.60068

In [57]:
valid=valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=(img,img),
    batch_size=b_s,
    class_mode="categorical",
    shuffle=False
)

Found 2058 images belonging to 15 classes.


In [22]:
test=valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=(img,img),
    batch_size=b_s,
    class_mode="categorical",
    shuffle=False
)

Found 2076 images belonging to 15 classes.


In [59]:
model = load_model(
    "/kaggle/input/plant-disease-detection/keras/default/1/my_model_checkpoint.keras",
    compile=False
)


In [60]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-05),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [61]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau, ModelCheckpoint

In [62]:
cb = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3),
    ModelCheckpoint("best_model_resume.keras",monitor="val_loss",save_best_only=True)
]

In [63]:
history = model.fit(
    train,
    epochs=60,         
    initial_epoch=38, 
    validation_data=valid,
    callbacks=cb
)


Epoch 39/60
516/516 ━━━━━━━━━━━━━━━━━━━━ 232s 442ms/step - accuracy: 0.9467 - loss: 0.1690 - val_accuracy: 0.9514 - val_loss: 0.1408 - learning_rate: 1.0000e-04
Epoch 40/60
516/516 ━━━━━━━━━━━━━━━━━━━━ 227s 439ms/step - accuracy: 0.9455 - loss: 0.1642 - val_accuracy: 0.9572 - val_loss: 0.1107 - learning_rate: 1.0000e-04
Epoch 41/60
516/516 ━━━━━━━━━━━━━━━━━━━━ 228s 442ms/step - accuracy: 0.9501 - loss: 0.1598 - val_accuracy: 0.9650 - val_loss: 0.0981 - learning_rate: 1.0000e-04
Epoch 42/60
516/516 ━━━━━━━━━━━━━━━━━━━━ 232s 449ms/step - accuracy: 0.9523 - loss: 0.1492 - val_accuracy: 0.9558 - val_loss: 0.1215 - learning_rate: 1.0000e-04
Epoch 43/60
516/516 ━━━━━━━━━━━━━━━━━━━━ 236s 458ms/step - accuracy: 0.9497 - loss: 0.1580 - val_accuracy: 0.9582 - val_loss: 0.1164 - learning_rate: 1.0000e-04
Epoch 44/60
516/516 ━━━━━━━━━━━━━━━━━━━━ 234s 454ms/step - accuracy: 0.9511 - loss: 0.1451 - val_accuracy: 0.9524 - val_loss: 0.1274 - learning_rate: 1.0000e-04
Epoch 45/60
516/516 ━━━━━━━━━━━━━━

In [13]:
m=load_model("/kaggle/input/best-model/keras/default/1/best_model_resume.keras")

In [26]:
m.evaluate(test)

65/65 ━━━━━━━━━━━━━━━━━━━━ 49s 749ms/step - accuracy: 0.9677 - loss: 0.0987


[0.12145479768514633, 0.960500955581665]

In [34]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_pred_probs = m.predict(test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_true = test.classes

class_names = list(test.class_indices.keys())

cm = confusion_matrix(y_true, y_pred)
print(cm)

print(classification_report(y_true, y_pred, target_names=class_names))


65/65 ━━━━━━━━━━━━━━━━━━━━ 47s 719ms/step
[[101   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 148   0   0   1   0   0   0   0   0   0   0   0   0   0]
 [  0   0  99   1   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   3  92   3   0   0   2   0   0   0   0   0   0   0]
 [  0   0   0   0  15   0   0   0   0   0   0   1   0   0   0]
 [  0   0   0   0   0 209   1   1   0   2   0   1   0   0   0]
 [  1   0   0   2   0   0  92   0   0   2   0   3   0   0   0]
 [  0   1   4   5   0   0   0 181   0   1   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1  90   4   0   1   0   0   0]
 [  2   0   6   0   0   0   2   0   0 165   0   1   0   2   0]
 [  0   0   0   0   0   0   0   0   0   0 155  11   0   2   1]
 [  0   0   0   0   0   0   0   1   0   1   4 131   0   1   3]
 [  1   0   0   0   0   3   0   0   0   0   0   0 318   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  38   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 160]]
            